# PHASE 2 PROJECT - MOVIE STUDIO BUSINESS INSIGHTS


## GROUP 4 PROJECT SUBMISSION

* Students names: Valentine Kweyu, Shem Nderitu, Mercy Barminga, Timothy Kamwilwa, Beatrice Kiilu, Nelson Muia, & Sharon Chebet.
* Student pace: Data Science/Part time 
* Scheduled project review date/time: 12th June 2025.
* Instructor name: Maryann Mwikali
* Blog post URL:


## SECTION 1: BUSINESS UNDERSTANDING

### Business Problem
The company wants to launch a new movie studio to produce original video content. However, the team lacks data-driven insights into what types of movies perform well in the market.


### Objective
Use exploratory data analysis (EDA) on movie data from various sources (Rotten Tomatoes, The Numbers, IMDB) to identify trends, patterns, and business opportunities in the film industry.

Provide 3 actionable recommendations to guide the new studio in film production.

### Stakeholders
- Head of the company's new movie studio
- Marketing team
- Content development team